In [132]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score

import statsmodels.api as sm

import matplotlib.pyplot as plt

# MPG dataset

## Data Dictionary

    1. mpg:           continuous
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)

In [36]:
df = pd.read_csv('auto-mpg.data')
df.drop(columns = 'junk', inplace = True)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70.0,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70.0,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70.0,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70.0,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70.0,1,ford torino


In [37]:
df.shape

(392, 9)

In [38]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight          float64
acceleration    float64
model_year      float64
origin            int64
car_name         object
dtype: object

## Features via Correlation

In [39]:
df.corr()[['mpg']].sort_values(by='mpg', ascending = False)

,mpg
mpg,1.000000
model_year,0.580541
origin,0.565209
acceleration,0.423329
cylinders,-0.777618
horsepower,-0.778427
displacement,-0.805127
weight,-0.832244


In [40]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

In [41]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     252.4
Date:                Fri, 10 Jan 2020   Prob (F-statistic):          2.04e-139
Time:                        11:39:51   Log-Likelihood:                -1023.5
No. Observations:                 392   AIC:                             2063.
Df Residuals:                     384   BIC:                             2095.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -17.2184      4.644     -3.707      0.000     -26.350      -8.087
cylinders       -0.4934      0.323     -1.526      0.128      -1.129       0.142
displacement     0.0199      0.008      2.647      0.008       0.005       0.035
horsepower      -0.0170      0.014     -1.230      0.220      -0.044       0.010
weight          -0.0065      0.001     -9.929      0.000      -0.008      -0.005
acceleration     0.0806      0.099      0.815      0.415      -0.114       0.275
model_year       0.7508      0.051     14.729      0.000       0.651       0.851
origin           1.4261      0.278      5.127      0.000       0.879       1.973
==============================================================================
Omnibus:                       31.906   Durbin-Watson:                   1.309
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.100
Skew:                           0.529   Prob(JB):                     2.95e-12
Kurtosis:                       4.460   Cond. No.                     8.59e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.59e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Creating Interaction Terms

In [42]:
X = df.drop(['mpg', 'car_name'], axis = 1)

In [43]:
poly = PolynomialFeatures()
X_poly = poly.fit_transform(X)
X_poly_features = poly.get_feature_names(X.columns)

In [44]:
X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

In [45]:
y = df['mpg']
X_poly = sm.add_constant(X_poly)

model = sm.OLS(y, X_poly).fit()

/Users/justinaugust/opt/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [46]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     86.99
Date:                Fri, 10 Jan 2020   Prob (F-statistic):          1.96e-152
Time:                        11:39:57   Log-Likelihood:                -918.86
No. Observations:                 392   AIC:                             1910.
Df Residuals:                     356   BIC:                             2053.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
1                           383.9269    100.585      3.817      0.000     186.110     581.743
cylinders                     8.4359      8.660      0.974      0.331      -8.595      25.467
displacement                 -0.5903      0.193     -3.057      0.002      -0.970      -0.211
horsepower                    0.2059      0.397      0.519      0.604      -0.575       0.987
weight                        0.0168      0.019      0.904      0.367      -0.020       0.053
acceleration                 -7.3501      2.586     -2.843      0.005     -12.435      -2.265
model_year                   -8.1099      2.230     -3.637      0.000     -12.495      -3.724
origin                      -20.8036      7.301     -2.849      0.005     -35.163      -6.444
cylinders^2                  -0.5122      0.451     -1.135      0.257      -1.400       0.376
cylinders displacement        0.0111      0.019      0.594      0.553      -0.026       0.048
cylinders horsepower          0.0021      0.026      0.080      0.936      -0.049       0.054
cylinders weight              0.0006      0.001      0.568      0.570      -0.002       0.003
cylinders acceleration        0.2072      0.174      1.189      0.235      -0.135       0.550
cylinders model_year         -0.1391      0.097     -1.430      0.154      -0.331       0.052
cylinders origin              0.4796      0.560      0.857      0.392      -0.621       1.580
displacement^2               -0.0001      0.000     -0.553      0.581      -0.001       0.000
displacement horsepower       0.0005      0.000      1.127      0.260      -0.000       0.001
displacement weight        1.163e-05   2.37e-05      0.491      0.624   -3.49e-05    5.82e-05
displacement acceleration     0.0006      0.004      0.146      0.884      -0.007       0.008
displacement model_year       0.0059      0.002      2.474      0.014       0.001       0.011
displacement origin           0.0189      0.021      0.884      0.377      -0.023       0.061
horsepower^2                 -0.0003      0.001     -0.431      0.667      -0.001       0.001
horsepower weight         -2.645e-05    4.4e-05     -0.601      0.548      -0.000    6.01e-05
horsepower acceleration      -0.0046      0.008     -0.570      0.569      -0.020       0.011
horsepower model_year        -0.0023      0.004     -0.552      0.581      -0.011       0.006
horsepower origin             0.0070      0.030      0.231      0.818      -0.052       0.066
weight^2                   2.339e-07   1.09e-06      0.215      0.830   -1.91e-06    2.37e-06
weight acceleration       -5.051e-06      0.000     -0.015      0.988      -0.001       0.001
weight model_year            -0.0003      0.000     -1.499      0.135      -0.001       0.000
weight origin                -0.0006      0.002     -0.375      0.708      

# Apply Regularization to Aid in Feature Selection
## Re-define X, y from original dataframe to drop the constant from `statsmodels`

In [94]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

## Re-create your polynomial features

In [95]:
poly = PolynomialFeatures()

X_poly = poly.fit_transform(X)
X_poly_features = poly.get_feature_names(X.columns)

X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

## Train, Test, Split

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state = 737)

## 👏🏻 Scale 👏🏻 Your 👏🏻 Data 👏🏻 

In [107]:
ss = StandardScaler()

In [108]:
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

## RidgeCV

In [109]:
ridge = RidgeCV()


ridge.fit(X_train_sc, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [110]:
ridge.score(X_train_sc,y_train)

0.8796072422873896

In [133]:
ridge.score(X_test_sc, y_test)

0.8799410577140421

In [135]:
cross_val_score(ridge, X_train_sc, y_train, cv = 5).mean()

0.8473555254510089

In [112]:
ridge.coef_

array([ 0.        , -1.1822314 , -1.72960207,  0.20692128, -1.94098065,
       -2.43574053,  0.15199258, -0.67741491,  0.0060215 ,  0.48688085,
        1.03625361,  2.02178937,  1.40141004, -2.1329727 ,  0.82137025,
        0.42394056,  0.13945055,  1.49925689, -0.18652645, -2.20724045,
        1.57430913,  1.4756173 ,  1.61500563, -1.31695291, -3.01509745,
       -1.21178957,  1.5395934 , -0.88370442, -3.83794385, -0.31419722,
        1.55898411,  0.56634201,  1.98187216,  3.91331553,  0.063782  ,
       -1.53807146])

In [176]:
pd.DataFrame(zip(X_poly_features, ridge.coef_), columns = ['feature', 'coefficient'] ).sort_values(by='coefficient')

,feature,coefficient
28,weight model_year,-3.837944
24,horsepower model_year,-3.015097
5,acceleration,-2.435741
19,displacement model_year,-2.207240
13,cylinders model_year,-2.132973
4,weight,-1.940981
2,displacement,-1.729602
35,origin^2,-1.538071
23,horsepower acceleration,-1.316953
25,horsepower origin,-1.211790


### How do we interpret these coefficients?

### What features might you remove? How would you make that decision?

## Lasso

In [144]:
lasso = LassoCV(max_iter = 10_000)
lasso = lasso.fit(X_train_sc, y_train)

In [121]:
lasso.coef_

array([ 0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.3500881 ,  0.        , -0.        ,  0.39373241,
        0.        ,  0.        ,  0.        , -0.29193781, -0.49049979,
        0.4065819 ,  1.21738567,  3.77100667, -1.91188114, -2.93243834,
       -0.31548885,  2.97453561, -0.        , -7.86494997,  0.        ,
        0.52758228,  0.        ,  1.49407715,  4.486823  ,  0.        ,
       -1.29158231])

In [122]:
lasso.score(X_train_sc,y_train)

0.8768432233942446

In [142]:
lasso.score(X_test_sc,y_test)

0.8763553653889857

In [145]:
cross_val_score(lasso, X_train_sc, y_train, cv = 5).mean()

0.8520710168638768

In [125]:
pd.DataFrame(zip(X_poly_features, lasso.coef_), columns = ['feature', 'coefficient'] ).sort_values(by='coefficient')

,feature,coefficient
28,weight model_year,-7.864950
24,horsepower model_year,-2.932438
23,horsepower acceleration,-1.911881
35,origin^2,-1.291582
19,displacement model_year,-0.490500
25,horsepower origin,-0.315489
18,displacement acceleration,-0.291938
31,acceleration model_year,0.000000
29,weight origin,0.000000
27,weight acceleration,-0.000000


### How do we interpret these coefficients?

### What features might you remove? How would you make that decision?

# Extra, Streeeeeeetch
## Recreating the `statsmodels` `.summary()` output to get p-values for your features

### _Liberally lifted from and based on https://stackoverflow.com/a/42677750_

In [130]:
from scipy import stats

In [173]:
def model_summary(estimator, X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 737)
    
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model = estimator()
    if LassoCV:
        model = estimator(max_iter = 10_000)
    model.fit(X_train_sc,y_train)
    
    print(f'Training Score: {model.score(X_train_sc,y_train)}')
    print(f'Testing Score: {model.score(X_test_sc,y_test)}')
    print(f'Cross Validation Score: {cross_val_score(model, X_train_sc, y_train).mean()}\n')
    
    
    params = np.append(model.intercept_,model.coef_)
    predictions = model.predict(X_train_sc)

    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
#     newX = np.append(np.ones((len(X),1)), X, axis=1)
#     MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    print(myDF3)

In [174]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

poly = PolynomialFeatures()

X_poly = poly.fit_transform(X)
X_poly_features = poly.get_feature_names(X.columns)

X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

In [175]:
model_summary(LassoCV, X_poly, y)

Training Score: 0.8768432233942446
Testing Score: 0.8763553653889857
Cross Validation Score: 0.8520710168638768

    Coefficients  Standard Errors     t values  Probabilites
0        23.5480              NaN          NaN           NaN
1         0.0000              NaN          NaN           NaN
2        -0.0000            7.921       -0.000         1.000
3        -0.0000            0.160       -0.000         1.000
4        -0.0000            0.156       -0.000         1.000
5        -0.0000            0.019       -0.000         1.000
6        -0.0000            2.792       -0.000         1.000
7         0.0000              NaN          NaN           NaN
8        -0.0000            8.555       -0.000         1.000
9         0.0000            0.475        0.000         1.000
10        0.0000            0.031        0.000         1.000
11        0.0000            0.024        0.000         1.000
12        0.3501            0.001      336.573         0.000
13        0.0000            0.098

/Users/justinaugust/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in sqrt
